## Importing Libraries

In [92]:
!pip install gradio==3.18.0 --q
!pip show gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.6 MB/s eta 0:00:00
Name: gradio
Version: 3.18.0
Summary: Python library for easily interacting with trained machine learning models
Home-page: 
Author: 
Author-email: Abubakar Abid <team@gradio.app>, Ali Abid <team@gradio.app>, Ali Abdalla <team@gradio.app>, Dawood Khan <team@gradio.app>, Ahsen Khaliq <team@gradio.app>, Pete Allen <team@gradio.app>, Ömer Faruk Özdemir <team@gradio.app>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiofiles, aiohttp, altair, fastapi, ffmpy, fsspec, httpx, jinja2, markdown-it-py, markupsafe, matplotlib, numpy, orjson, pandas, pillow, pycryptodome, pydantic, pydub, python-multipart, pyyaml, requests, typing-extensions, uvicorn, websockets
Required-by: 


In [ ]:
!pip install langchain openai chromadb tiktoken pypdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.5/249.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.6/922.6 kB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
    

In [143]:
import os 
os.environ["OPENAI_API_KEY"] = ''

In [82]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.document_loaders import PyPDFLoader

## Q&A

In [144]:
llm = OpenAI()
loader = PyPDFLoader("/content/Operations Management.pdf")
document = loader.load()
chain = load_qa_chain(llm=OpenAI(), chain_type="map_reduce")

In [145]:
def answer_question(document,query):
  result = chain.run({"input_documents": document, "question":query})
  return result

## Agent

In [84]:
tools = [Tool(
        name = "Pdf QA System",
        func = answer_question,
        description="useful for when you need to answer questions about pdf. Input should be a fully formed question."
    )]
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [60]:
while True:
  query=input("Enter Your Question(n to exit):")
  if query=='n':
    break
  agent_chain.run(query)

Enter Your Question(n to exit):what is operations management?


> Entering new AgentExecutor chain...
I need to find a definition or explanation of operations management.
Action: Pdf QA System
Action Input: "What is operations management?"
Observation:  Operations management is the process of planning, organizing, allocating activities, staffing, leading, and making decisions over all processes in the organization. It determines the direction of such decisions with consideration to cost, distance and time factors from both the source of raw materials and the organization’s customers. It also outlines the activities to be done and when such activities are to be undertaken. Indicators of operations performance include cost, quality, speed, dependability and flexibility. The benefits of operations management include effective utilization of scarce human and material resources, reduced production costs, and ensured satisfaction of customer's needs.
Thought:I have a good understanding of wh

## Front end using gradio

Unfortunately,due to restrictions on OpenAI requests, I cannot make any further request, so am not able to show the front end working.

In [152]:
# Front end web app
import gradio as gr
with gr.Blocks() as demo:
    upload_button = gr.UploadButton("Upload Pdf", file_types=[".pdf"])
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")
    chat_history = []      
    def user(user_message, history):
        response = answer_question(document,user_message) #agent_chain.run(user_message)#
        history.append((user_message, response))
        return gr.update(value=""), history
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False)
    clear.click(lambda: None, None, chatbot, queue=False)       

In [153]:
demo.launch(debug = True) 

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
